# Import libraries

In [ ]:
import os, random
import pandas as pd
import numpy as np
import torch
from easydict import EasyDict
from mne.decoding import CSP
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from dataloader import BCICompet2aIV

# Configs

In [ ]:
args = EasyDict({
    'BASE_PATH': './data/BCI_Competition_IV/BCI_Competition_IV_2a',
    'SEED': 42,
    
    'num_subjects': 9,
    'EPOCHS': 100,
    'lr': 1e-3
})

# Set seed

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.fastest = True
    
seed_everything(args.SEED)

# Training

In [ ]:
acc = []
test_preds = []

for target_subject in range(args.num_subjects):
    print('='*30)
    print(f'LOG >>> Target subject: {target_subject}')
    print('='*30)
    
    train_dataset = BCICompet2aIV(base_path=args.BASE_PATH,
                                  target_subject=target_subject,
                                  is_subject_independent=True,
                                  is_val=False)
    val_dataset = BCICompet2aIV(base_path=args.BASE_PATH,
                                target_subject=target_subject, 
                                is_subject_independent=True, 
                                is_val=True)
    test_dataset = BCICompet2aIV(base_path=args.BASE_PATH,
                                 target_subject=target_subject,
                                 is_subject_independent=True,
                                 is_test=True)
    
    ### Change csp&lda to neural net based model ###
    X_train, y_train = train_dataset.data, train_dataset.label
    X_val, y_val = val_dataset.data, val_dataset.label
    X_test = test_dataset.data
    
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
    ss = preprocessing.StandardScaler()
    
    X_train_csp = ss.fit_transform(csp.fit_transform(X_train.squeeze(), y_train))
    X_val_csp = ss.transform(csp.transform(X_val.squeeze()))
    X_test_csp = ss.transform(csp.transform(X_test.squeeze()))
    
    clf = LinearDiscriminantAnalysis()
    clf.fit(X_train_csp, y_train)
    
    y_pred = clf.predict(X_val_csp)
    y_test_pred = clf.predict(X_test_csp)
    #################################################
    
    acc.append(accuracy_score(y_val, y_pred))
    test_preds.append(y_test_pred)
    
    print('='*30)
    print(f'LOG >>> {target_subject} acc. : {acc[-1]:.4f}')
    print('='*30)

In [ ]:
acc

# Submission

In [ ]:
sample_submission = pd.DataFrame(test_preds).T
sample_submission.to_csv('sample_submission.csv', index=False)
sample_submission.head()